![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.png)
# Pipeline 8 Notebook - AutoAI Notebook v1.15.6

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimum results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a>. 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for getting data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.8 and scikit-learn 0.23.2.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data](#read)<br>
&nbsp;&nbsp;[Train and test data split](#split)<br>
&nbsp;&nbsp;[Make pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
**[Next steps](#next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-libs,
 - scikit-learn,
 - lightgbm.


In [1]:
!pip install -U autoai-libs==1.12.13 
!pip install -U scikit-learn==0.23.2 
!pip install -U lightgbm==3.1.1 
!pip install pandas 
!pip install lale==0.5.3
!pip install joblib



Requirement already up-to-date: autoai-libs==1.12.13 in c:\users\kacpe\desktop\przedmioty zawodowe\aplikacje desktopowe\venv\lib\site-packages (1.12.13)


You should consider upgrading via the 'c:\users\kacpe\desktop\przedmioty zawodowe\aplikacje desktopowe\venv\scripts\python.exe -m pip install --upgrade pip' command.


Requirement already up-to-date: scikit-learn==0.23.2 in c:\users\kacpe\desktop\przedmioty zawodowe\aplikacje desktopowe\venv\lib\site-packages (0.23.2)


You should consider upgrading via the 'c:\users\kacpe\desktop\przedmioty zawodowe\aplikacje desktopowe\venv\scripts\python.exe -m pip install --upgrade pip' command.


Requirement already up-to-date: lightgbm==3.1.1 in c:\users\kacpe\desktop\przedmioty zawodowe\aplikacje desktopowe\venv\lib\site-packages (3.1.1)


You should consider upgrading via the 'c:\users\kacpe\desktop\przedmioty zawodowe\aplikacje desktopowe\venv\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\kacpe\desktop\przedmioty zawodowe\aplikacje desktopowe\venv\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\kacpe\desktop\przedmioty zawodowe\aplikacje desktopowe\venv\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\kacpe\desktop\przedmioty zawodowe\aplikacje desktopowe\venv\scripts\python.exe -m pip install --upgrade pip' command.


<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('C:/Users/kacpe/Desktop/german_credit_data_biased_training.csv', encoding = "utf-8")
df_X = df.drop('Risk', axis=1)
df_y = df.Risk
train_X, test_X, train_y, test_y = train_test_split(df_X, df_y, test_size=0.1, random_state=33, stratify=df_y)

In [22]:
train_X.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,CurrentResidenceDuration,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker
4144,less_0,4,credits_paid_to_date,car_new,1269,less_100,1_to_4,3,female,none,3,real_estate,31,bank,rent,1,unskilled,1,none,yes
4707,less_0,16,credits_paid_to_date,furniture,1130,less_100,less_1,1,female,none,1,real_estate,30,stores,rent,1,skilled,1,none,yes
3455,less_0,14,all_credits_paid_back,furniture,250,less_100,4_to_7,3,male,none,2,real_estate,24,none,own,1,skilled,1,none,yes
3106,less_0,4,all_credits_paid_back,car_new,250,100_to_500,1_to_4,2,female,none,2,real_estate,33,bank,rent,1,unemployed,1,none,yes
2720,greater_200,46,prior_payments_delayed,furniture,2546,500_to_1000,greater_7,4,male,guarantor,2,savings_insurance,36,stores,free,1,unskilled,1,yes,yes


In [23]:
train_y.head()

4144    No Risk
4707    No Risk
3455    No Risk
3106    No Risk
2720       Risk
Name: Risk, dtype: object

<a id="preview_model_to_python_code"></a>
## Make pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [24]:
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TA1
import autoai_libs.utils.fc_methods
from autoai_libs.cognito.transforms.transform_utils import FS1
from autoai_libs.cognito.transforms.transform_utils import TAM
from sklearn.decomposition import PCA
from lightgbm.sklearn import LGBMClassifier

#### Pre-processing & Estimator.

In [25]:
numpy_column_selector_0 = NumpyColumnSelector(
    columns=[
        0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
    ]
)
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "char_str", "int_num", "char_str", "char_str", "char_str", "char_str",
        "int_num", "char_str", "char_str", "int_num", "char_str", "int_num",
        "char_str", "char_str", "int_num", "char_str", "int_num", "char_str",
        "char_str",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [],
    ],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"),
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    strategy="most_frequent",
    missing_values=float("nan"),
    sklearn_version_family="23",
)
cat_encoder = CatEncoder(
    encoding="ordinal",
    categories="auto",
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="23",
)
pipeline_0 = make_pipeline(
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
numpy_column_selector_1 = NumpyColumnSelector(columns=[4])
float_str2_float = FloatStr2Float(
    dtypes_list=["int_num"], missing_values_reference_list=[]
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
num_imputer = NumImputer(strategy="median", missing_values=float("nan"))
opt_standard_scaler = OptStandardScaler(
    num_scaler_copy=None,
    num_scaler_with_mean=None,
    num_scaler_with_std=None,
    use_scaler_flag=False,
)
pipeline_1 = make_pipeline(
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 4,
    ],
)
ta1 = TA1(
    fun=np.log,
    name="log",
    datatypes=["numeric"],
    feat_constraints=[
        autoai_libs.utils.fc_methods.is_positive,
        autoai_libs.utils.fc_methods.is_not_categorical,
    ],
    col_names=[
        "CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose",
        "LoanAmount", "ExistingSavings", "EmploymentDuration",
        "InstallmentPercent", "Sex", "OthersOnLoan",
        "CurrentResidenceDuration", "OwnsProperty", "Age", "InstallmentPlans",
        "Housing", "ExistingCreditsCount", "Job", "Dependents", "Telephone",
        "ForeignWorker",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 20),
    additional_col_count_to_keep=20,
    ptype="classification",
)
tam = TAM(
    tans_class=PCA(),
    name="pca",
    col_names=[
        "CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose",
        "LoanAmount", "ExistingSavings", "EmploymentDuration",
        "InstallmentPercent", "Sex", "OthersOnLoan",
        "CurrentResidenceDuration", "OwnsProperty", "Age", "InstallmentPlans",
        "Housing", "ExistingCreditsCount", "Job", "Dependents", "Telephone",
        "ForeignWorker", "log(LoanDuration)", "log(LoanAmount)", "log(Age)",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 20),
    additional_col_count_to_keep=20,
    ptype="classification",
)
lgbm_classifier = LGBMClassifier(
    class_weight="balanced",
    colsample_bytree=0.6630873122491916,
    learning_rate=0.7253594972455494,
    min_child_samples=19,
    min_child_weight=0.00424841899293295,
    n_estimators=640,
    num_leaves=32,
    random_state=33,
    reg_alpha=0.00024836917720855287,
    reg_lambda=0.7211028773552123,
    subsample=0.9751191846731722,
    subsample_freq=5,
)


#### Pipeline.

In [26]:
pipeline = make_pipeline(
    union, numpy_permute_array, ta1, fs1_0, tam, fs1_1, lgbm_classifier
)

Pipeline visualisation - unfotunatelly it doesn't work, we found bug for Windows. We need to develop how to fix it.

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [27]:
from sklearn.metrics import get_scorer

scorer = get_scorer("accuracy")

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [28]:
pipeline.fit(train_X.values, train_y.values);

c:\users\kacpe\desktop\przedmioty zawodowe\aplikacje desktopowe\venv\lib\site-packages\sklearn\base.py:693: RuntimeWarning: divide by zero encountered in log
  return self.fit(X, y, **fit_params).transform(X)


<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [29]:
score = scorer(pipeline, test_X.values, test_y.values)
print(score)

0.776


c:\users\kacpe\desktop\przedmioty zawodowe\aplikacje desktopowe\venv\lib\site-packages\sklearn\pipeline.py:407: RuntimeWarning: divide by zero encountered in log
  Xt = transform.transform(Xt)


In [30]:
pipeline.predict(test_X.values)

c:\users\kacpe\desktop\przedmioty zawodowe\aplikacje desktopowe\venv\lib\site-packages\sklearn\pipeline.py:407: RuntimeWarning: divide by zero encountered in log
  Xt = transform.transform(Xt)


array(['No Risk', 'Risk', 'No Risk', 'No Risk', 'No Risk', 'Risk', 'Risk',
       'No Risk', 'Risk', 'No Risk', 'No Risk', 'No Risk', 'No Risk',
       'Risk', 'No Risk', 'Risk', 'Risk', 'No Risk', 'No Risk', 'No Risk',
       'No Risk', 'No Risk', 'No Risk', 'Risk', 'No Risk', 'Risk', 'Risk',
       'No Risk', 'No Risk', 'Risk', 'No Risk', 'No Risk', 'No Risk',
       'No Risk', 'Risk', 'Risk', 'No Risk', 'No Risk', 'No Risk',
       'No Risk', 'No Risk', 'No Risk', 'No Risk', 'Risk', 'Risk',
       'No Risk', 'Risk', 'No Risk', 'Risk', 'Risk', 'No Risk', 'No Risk',
       'No Risk', 'No Risk', 'Risk', 'No Risk', 'No Risk', 'No Risk',
       'Risk', 'Risk', 'No Risk', 'Risk', 'No Risk', 'No Risk', 'No Risk',
       'No Risk', 'Risk', 'No Risk', 'No Risk', 'No Risk', 'Risk',
       'No Risk', 'No Risk', 'No Risk', 'No Risk', 'Risk', 'Risk',
       'No Risk', 'No Risk', 'No Risk', 'No Risk', 'No Risk', 'No Risk',
       'Risk', 'No Risk', 'No Risk', 'Risk', 'Risk', 'No Risk', 'Risk',
  